In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.12 already installed.


## Download dataset

Original VLBA dataset gotten from https://casaguides.nrao.edu/index.php?title=VLBA_Basic_Phase-referencing_Calibration_and_Imaging.

Reduced the data using:

```Python
mstransform(vis='VLBA_TL016B.ms/',outputvis='VLBA_TL016B_split_lsrk.ms',spw='0:0~5,1:0~5',field='0,1', timerange='2022/02/21/06:14:00~2022/02/21/07:55:45.00', regridms=True,outframe='lsrk',datacolumn='all')
```




In [1]:
import graphviper

graphviper.utils.data.download(file="VLBA_TL016B_split_lsrk.ms")

VLBA_TL016B_split_lsrk.ms.zip:   0%|          | 0.00/8.96M [00:00<?, ?iB/s]

## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "VLBA_TL016B_split_lsrk.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [4]:
ps_name = outfile

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_name=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency
0,VLBA_TL016B_split_lsrk_ddi_1_intent_None_field...,1,None,1,J1154+6022,5.068199e+09,5.070699e+09
1,VLBA_TL016B_split_lsrk_ddi_1_intent_None_field...,1,None,0,4C39.25,5.068199e+09,5.070699e+09
2,VLBA_TL016B_split_lsrk_ddi_0_intent_None_field...,0,None,0,4C39.25,5.004196e+09,5.006697e+09
3,VLBA_TL016B_split_lsrk_ddi_0_intent_None_field...,0,None,1,J1154+6022,5.004196e+09,5.006697e+09


In [5]:
ps.keys()

dict_keys(['VLBA_TL016B_split_lsrk_ddi_1_intent_None_field_id_1', 'VLBA_TL016B_split_lsrk_ddi_1_intent_None_field_id_0', 'VLBA_TL016B_split_lsrk_ddi_0_intent_None_field_id_0', 'VLBA_TL016B_split_lsrk_ddi_0_intent_None_field_id_1'])

In [6]:
ps['VLBA_TL016B_split_lsrk_ddi_1_intent_None_field_id_1']

<xarray.Dataset>
Dimensions:                     (time: 540, baseline_id: 55, frequency: 6,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int64 dask.array<chunksize=(55,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int64 dask.array<chunksize=(55,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 0 1 2 3 4 ... 50 51 52 53 54
  * frequency                   (frequency) float64 5.068e+09 ... 5.071e+09
  * polarization                (polarization) <U2 'RR' 'LL'
  * time                        (time) float64 1.645e+09 1.645e+09 ... 1.645e+09
  * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(20, 55), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 dask.array<chunksize=(20, 55), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 dask.array<chunksize=(20, 55, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(20, 55, 1, 2), meta=np.ndarray>
Attributes:
    data_groups:  {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VISI...
    ddi:          1
    field_info:   {'code': '', 'delay_direction': {'attrs': {'frame': 'FK5', ...
    intent:       None
    antenna_xds:  <xarray.Dataset>\nDimensions:        (antenna_id: 10, xyz_l...

In [7]:
ps['VLBA_TL016B_split_lsrk_ddi_1_intent_None_field_id_1'].attrs['antenna_xds']

<xarray.Dataset>
Dimensions:        (antenna_id: 10, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 0 1 2 3 4 5 6 7 8 9
    mount          (antenna_id) <U6 dask.array<chunksize=(10,), meta=np.ndarray>
    name           (antenna_id) <U2 dask.array<chunksize=(10,), meta=np.ndarray>
    station        (antenna_id) <U2 dask.array<chunksize=(10,), meta=np.ndarray>
    type           (antenna_id) <U12 dask.array<chunksize=(10,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 dask.array<chunksize=(10, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 dask.array<chunksize=(10, 3), meta=np.ndarray>